<a href="https://colab.research.google.com/github/CakeNuthep/Super_AI/blob/master/LSTM/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# cell 1
%tensorflow_version

Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


In [ ]:
# cell 2
!wget http://www.manythings.org/anki/tha-eng.zip

--2020-09-22 03:04:57--  http://www.manythings.org/anki/tha-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.173.198, 104.24.109.196, 104.24.108.196, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.173.198|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105315 (103K) [application/zip]
Saving to: ‘tha-eng.zip’

tha-eng.zip         100%[===================>] 102.85K  --.-KB/s    in 0.1s    

2020-09-22 03:04:58 (775 KB/s) - ‘tha-eng.zip’ saved [105315/105315]



In [ ]:
# cell 3
!unzip tha-eng.zip

Archive:  tha-eng.zip
  inflating: _about.txt              
  inflating: tha.txt                 


In [ ]:
# cell 4
!head tha.txt

Hi.	สวัสดี	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #499079 (ojiisan)
Run.	วิ่ง	CC-BY 2.0 (France) Attribution: tatoeba.org #4008918 (JSakuragi) & #8676110 (pastelite)
Who?	ใคร?	CC-BY 2.0 (France) Attribution: tatoeba.org #2083030 (CK) & #7878433 (pastelite)
Wow!	ว้าว!	CC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #7878663 (pastelite)
Fire!	ยิง!	CC-BY 2.0 (France) Attribution: tatoeba.org #1829639 (Spamster) & #8675907 (pastelite)
Help!	ช่วยด้วย!	CC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #7784241 (pastelite)
Jump!	กระโดด!	CC-BY 2.0 (France) Attribution: tatoeba.org #1102981 (jamessilver) & #7784245 (pastelite)
Jump.	กระโดด	CC-BY 2.0 (France) Attribution: tatoeba.org #631038 (Shishir) & #7784244 (pastelite)
Stop!	หยุด!	CC-BY 2.0 (France) Attribution: tatoeba.org #448320 (FeuDRenais) & #7784240 (pastelite)
Wait.	รอก่อน	CC-BY 2.0 (France) Attribution: tatoeba.org #3048304 (camilozeta) & #7784248 (pastelite)


In [ ]:
# cell 5
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.optimizers import RMSprop
import numpy as np

In [ ]:
# cell 6
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'tha.txt'

In [ ]:
# cell 7
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set() # unique characters in input
target_characters = set() # unique characters in target

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

for line in lines[: min(num_samples, len(lines) - 1)]: # train only on num_samples
    input_text, target_text, _ = line.split('\t') # split by tabs and get only the first two strings
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n' # pad \n to use as end of sequence
    input_texts.append(input_text)
    target_texts.append(target_text)

    # populate the sets for both the input and target
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [ ]:
# cell 8
print(input_texts[0:5])
print(target_texts[0:5])

['Hi.', 'Run.', 'Who?', 'Wow!', 'Fire!']
['\tสวัสดี\n', '\tวิ่ง\n', '\tใคร?\n', '\tว้าว!\n', '\tยิง!\n']


In [ ]:
# cell 9
input_characters = sorted(list(input_characters)) # convert the set to list and sort
target_characters = sorted(list(target_characters)) # convert the set to list and sort
num_encoder_tokens = len(input_characters) # number of unique characters in input
num_decoder_tokens = len(target_characters) # number of unique characters in target

max_encoder_seq_length = max([len(txt) for txt in input_texts]) # find the longest sentence
max_decoder_seq_length = max([len(txt) for txt in target_texts]) # find the longest sentence

In [ ]:
# cell 10
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 2887
Number of unique input tokens: 74
Number of unique output tokens: 98
Max sequence length for inputs: 100
Max sequence length for outputs: 119


In [ ]:
# cell 11
# dicts that map char to int
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [ ]:
print(encoder_input_data.shape) # batch, time, features (each char is one feature, the feature dimension is one-hot)

(2887, 100, 74)


In [ ]:
print(input_token_index)
print(target_token_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, '?': 22, 'A': 23, 'B': 24, 'C': 25, 'D': 26, 'E': 27, 'F': 28, 'G': 29, 'H': 30, 'I': 31, 'J': 32, 'K': 33, 'L': 34, 'M': 35, 'N': 36, 'O': 37, 'P': 38, 'R': 39, 'S': 40, 'T': 41, 'U': 42, 'V': 43, 'W': 44, 'Y': 45, 'a': 46, 'b': 47, 'c': 48, 'd': 49, 'e': 50, 'f': 51, 'g': 52, 'h': 53, 'i': 54, 'j': 55, 'k': 56, 'l': 57, 'm': 58, 'n': 59, 'o': 60, 'p': 61, 'q': 62, 'r': 63, 's': 64, 't': 65, 'u': 66, 'v': 67, 'w': 68, 'x': 69, 'y': 70, 'z': 71, '₂': 72, '€': 73}
{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, '%': 5, '(': 6, ')': 7, ',': 8, '.': 9, '/': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, '?': 22, 'F': 23, 'H': 24, 'O': 25, 'a': 26, 'b': 27, 'c': 28, 'e': 29, 'k': 30, 'o': 31, 'ก': 32, 'ข': 33, 'ค': 34, 'ฆ': 35, 'ง': 36, 'จ': 37, 'ฉ': 38,

In [ ]:
# cell 14*
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)): # zip the two lists together so can iterate over them at the same time

    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1. # make the position of the char 1 (one hot vector style)
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1. # make rows after sequence finished all = 1 (t+1 till the end) to mark the sequence had already ended. Basically use space as padding

    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1. # -1 is shift to the right 1 unit in time, like math
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1. # same as above
    decoder_target_data[i, t:, target_token_index[' ']] = 1. # save as above

In [ ]:
print(encoder_input_data[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [ ]:
# cell 16*
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens)) # None in the first dim is not for batch, it is for time index, so model can accept inputs with different length
encoder = LSTM(latent_dim, return_state=True) # return the last state (NOT the same as return_sequence)
encoder_outputs, state_h, state_c = encoder(encoder_inputs) # return both the cell state and the regular h state
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# cell 17
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# cell 18*
# print summary
model.compile(optimizer=RMSprop(learning_rate=0.0005, momentum=0.9), loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 74)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 98)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 338944      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  363520      input_2[0][0]                    
                                                                 lstm[0][1]            

In [ ]:
# train and save
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.5)
# Save model
model.save('s2s.h5')

Epoch 1/100
23/23 [==============================] - 1s 45ms/step - loss: 0.1575 - accuracy: 0.9564 - val_loss: 0.4196 - val_accuracy: 0.8974
Epoch 2/100
23/23 [==============================] - 1s 35ms/step - loss: 0.1524 - accuracy: 0.9582 - val_loss: 0.4242 - val_accuracy: 0.8968
Epoch 3/100
23/23 [==============================] - 1s 34ms/step - loss: 0.1490 - accuracy: 0.9589 - val_loss: 0.4391 - val_accuracy: 0.8936
Epoch 4/100
23/23 [==============================] - 1s 34ms/step - loss: 0.1464 - accuracy: 0.9599 - val_loss: 0.4389 - val_accuracy: 0.8943
Epoch 5/100
23/23 [==============================] - 1s 35ms/step - loss: 0.1445 - accuracy: 0.9603 - val_loss: 0.4463 - val_accuracy: 0.8924
Epoch 6/100
23/23 [==============================] - 1s 35ms/step - loss: 0.1422 - accuracy: 0.9609 - val_loss: 0.4465 - val_accuracy: 0.8924
Epoch 7/100
23/23 [==============================] - 1s 35ms/step - loss: 0.1398 - accuracy: 0.9618 - val_loss: 0.4531 - val_accuracy: 0.8911
Epoch 

# Inference Mode

In [ ]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, # list concatenation, not arithmetic add
    [decoder_outputs] + decoder_states) # also output its own states (both c and h)

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: ฉันเป็นคนอาศิที่ฉัน

-
Input sentence: Run.
Decoded sentence: ฉันเป็นคนอาศิที่ฉัน

-
Input sentence: Who?
Decoded sentence: ฉันเป็นคนอาศิที่ฉัน

-
Input sentence: Wow!
Decoded sentence: ฉันเป็นคนอาศิที่ฉัน

-
Input sentence: Fire!
Decoded sentence: ฉันเป็นคนอาศิที่ฉัน

-
Input sentence: Help!
Decoded sentence: ฉันเป็นคนอาศิที่ฉัน

-
Input sentence: Jump!
Decoded sentence: ฉันเป็นคนอาศิที่ฉัน

-
Input sentence: Jump.
Decoded sentence: ฉันเป็นคนอาศิที่ฉัน

-
Input sentence: Stop!
Decoded sentence: ฉันเป็นคนอาศิที่ฉัน

-
Input sentence: Wait.
Decoded sentence: ฉันเป็นคนอาศิที่ฉัน

-
Input sentence: Go on.
Decoded sentence: ฉันเป็นคนอาศิที่ฉัน

-
Input sentence: Hello!
Decoded sentence: ฉันเป็นคนอาศิที่ฉัน

-
Input sentence: Hurry!
Decoded sentence: ฉันเป็นคนอาศิที่ฉัน

-
Input sentence: I see.
Decoded sentence: ฉันเป็นคนอาศิที่ฉัน

-
Input sentence: I try.
Decoded sentence: ฉันเป็นคนอาศิที่ฉัน

-
Input sentence: I won!
Decoded sentence: ฉันเป็นคนอาศ